# **PARTIE 2. Combinaison de caractéristiques**

# **Instruction 2.1. : Importation des librairies**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.gridspec as gridspec
from matplotlib.patches import ConnectionPatch, Rectangle
import matplotlib as mpl
import cv2
import os
from google.colab import files
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.display import HTML
from base64 import b64encode
import numpy as np
import glob
import imageio

# **Question 2.1 : Compléter pour avoir les filtres sur une image en niveau de gris**

In [ ]:
# A compléter ...
# N'oubliez pas de lire l'image ...
rgb_image = #########
gray = #######

gauss_gray = ########
median = #######
bilateral = #########
Sobelx = ########
Canny = #########

# **Instruction 2.2. : Fonctions pour transformer en Tensor et en Numpy**

In [ ]:
#les features maps actuellement sont en numpy 2D (H,W) et nous avons besoin d'un Tensor 4D : [1,1,H,W]
def np2torch_1ch(arr2d: np.ndarray, device="cpu") -> torch.Tensor:
    """np 2D (0..255 ou 0..1) -> torch [1,1,H,W] float32 0..1"""
    a = arr2d.astype(np.float32)
    if a.max() > 1.5: a = a / 255.0
    a = np.clip(a, 0.0, 1.0)
    return torch.from_numpy(a)[None, None].to(device)  # [1,1,H,W]

def norm01_np_from_torch(t: torch.Tensor) -> np.ndarray:
    """torch [1,1,H,W] -> np 2D normalisée 0..1 pour imshow"""
    a = t.detach().cpu().squeeze().numpy().astype(np.float32)
    mn, mx = a.min(), a.max()
    return np.zeros_like(a) if mx - mn < 1e-12 else (a - mn) / (mx - mn)

# **Question 2.2. : Compléter le dictionnaire des cartes de caractéristiques**

In [ ]:
# dictionnaire filtres -> images
feature_maps_dict = {
    "Gaussian (3x3)" : #########,
    "Median (3x3)" : #########,
    "Bilateral (21)" : #########,
    "Sobel X (5x5)" : np.abs(#########),
    "Canny" : #########
}

feature_maps = []

for k, v in feature_maps_dict.items():
    feature_maps.append(np2torch_1ch(v))

titles = ["Gaussian (3x3)", "Median (3x3)", "Bilateral (21)", "Sobel X (5x5)", "Canny"]


# **Instruction 2.3. : Affichage**

In [ ]:
n = len(feature_maps)
fig, axs = plt.subplots(1, n+1, figsize=(3*(n+1), 3))
axs[0].imshow(rgb_image); axs[0].set_title("Input"); axs[0].axis("off")
for i, fm in enumerate(feature_maps, 1):
    axs[i].imshow(norm01_np_from_torch(fm), cmap="gray"); axs[i].set_title(titles[i-1]); axs[i].axis("off")
plt.show()

# **Question 2.3. : Application Max Pooling et visualisation**

In [ ]:
# TODO : créer un MaxPool2d(kernel_size=2, stride=2)
###########

# TODO : appliquer le pool à chaque feature -> reduced_maps
reduced_maps = #######   # liste de [1,1,h_i,w_i]

# Visuel Étape 1
fig, axs = plt.subplots(1, n, figsize=(3*n, 3))
for i, rm in enumerate(reduced_maps):
    axs[i].imshow(norm01_np_from_torch(rm), cmap="gray"); axs[i].set_title("MaxPooling " + titles[i]), axs[i].axis("off")
plt.show()

# **Question 2.4. : Concaténation des cartes de caractéristiques**

In [ ]:
# On aligne d'abord toutes les reduced maps à la plus petite taille, puis on concatène verticalement.

import torch.nn.functional as F

# 2.1 Tailles et plus petite taille
sizes = [rm.shape[-2:] for rm in reduced_maps]   # [(h_i, w_i)]
min_h = min(h for h, w in sizes)
min_w = min(w for h, w in sizes)

# 2.2 Alignement (si déjà toutes à la même taille, on évite l'interpolation)
if all((h == min_h and w == min_w) for (h, w) in sizes):
    reduced_aligned = reduced_maps
else:
    reduced_aligned = [
        F.interpolate(rm, size=(min_h, min_w), mode="bilinear", align_corners=False)
        for rm in reduced_maps
    ]  # liste de [1,1,min_h,min_w]

# 2.3 Concat vertical -> une grande image [1,1, n*min_h, min_w]
stack_viz = torch.cat(reduced_aligned, dim=-2)

# Visuel Étape 2
plt.figure(figsize=(5, 12))
plt.imshow(norm01_np_from_torch(stack_viz), cmap="gray", aspect="auto")
plt.title(f"STACK vertical des reduced alignées\nsize={tuple(stack_viz.shape[-2:])}")
plt.axis("off")
plt.show()

# **Question 2.5. : Aplatir pour avoir un tensor 1D et afficher ses caractéristiques**

In [ ]:
# TODO : créer le vecteur par aplatissement du stack
fusion_vector = ##########
N = ############
print("Shape of fusion vector:", ######## )
print("Length of fusion vector:", ####### )

# **Instruction 2.4. : Visuel complet**

In [ ]:

import matplotlib.gridspec as gridspec
from matplotlib.patches import ConnectionPatch, Rectangle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

def arrow_with_label(ax_from, ax_to, text="", color="black", lw=1.4, text_offset=(0,0.05)):
    con = ConnectionPatch(
        xyA=(1.0, 0.5), coordsA=ax_from.transAxes,
        xyB=(0.0, 0.5), coordsB=ax_to.transAxes,
        arrowstyle="->", shrinkA=0, shrinkB=0,
        mutation_scale=12, lw=lw, color=color,
    )
    ax_to.add_artist(con)
    fig = ax_from.figure
    fbb = ax_from.get_position(); tbb = ax_to.get_position()
    x0, y0 = fbb.x1, (fbb.y0+fbb.y1)/2
    x1, y1 = tbb.x0, (tbb.y0+tbb.y1)/2
    xm, ym = (x0+x1)/2, (y0+y1)/2
    fig.text(xm + text_offset[0], ym + text_offset[1], text,
             ha="center", va="center", fontsize=9, color=color)

def arrow(ax_from, ax_to, color="black", lw=1.4):
    con = ConnectionPatch(
        xyA=(1.0, 0.5), coordsA=ax_from.transAxes,
        xyB=(0.0, 0.5), coordsB=ax_to.transAxes,
        arrowstyle="->", shrinkA=0, shrinkB=0,
        mutation_scale=12, lw=lw, color=color,
    )
    ax_to.add_artist(con)

# --- Normalisation globale 0..255 basée sur TOUT le stack ---
gmin = float(stack_viz.min().item())
gmax = float(stack_viz.max().item())
den  = max(gmax - gmin, 1e-12)

def to_u8_from_stack_global(t):
    a = t.detach().cpu().squeeze().float().numpy()
    a = (a - gmin) / den * 255.0
    return np.clip(a, 0, 255).astype(np.uint8)

# helper titres
if 'titles' not in globals() or not isinstance(titles, (list, tuple)) or len(titles) != n:
    titles = [f"F{i+1}" for i in range(n)]

fig = plt.figure(figsize=(16, 9))
gs = gridspec.GridSpec(nrows=n, ncols=5, width_ratios=[1.2, 1.0, 1.0, 1.2, 0.8],
                       wspace=0.45, hspace=0.25)

# Col 0: Input
ax0 = fig.add_subplot(gs[:, 0])
if 'img_rgb' in globals() and isinstance(img_rgb, np.ndarray) and img_rgb.ndim == 3:
    ax0.imshow(img_rgb)
else:
    ax0.imshow(to_u8_from_stack_global(feature_maps[0]), cmap="gray", vmin=0, vmax=255)
ax0.set_title("Input"); ax0.axis("off")

# Col 1: Features
ax_fms = []
for i in range(n):
    ax = fig.add_subplot(gs[i, 1])
    ax.imshow(to_u8_from_stack_global(feature_maps[i]), cmap="gray", vmin=0, vmax=255)
    ax.set_title(titles[i], fontsize=9); ax.axis("off")
    ax_fms.append(ax)

# Col 2: MaxPool
ax_rms = []
for i in range(n):
    ax = fig.add_subplot(gs[i, 2])
    ax.imshow(to_u8_from_stack_global(reduced_maps[i]), cmap="gray", vmin=0, vmax=255)
    ax.set_title(f"MaxPool {i+1}", fontsize=9); ax.axis("off")
    ax_rms.append(ax)

# Col 3: STACK
axS = fig.add_subplot(gs[:, 3])
stack_u8 = to_u8_from_stack_global(stack_viz)
axS.imshow(stack_u8, cmap="gray", vmin=0, vmax=255, aspect="auto")
axS.set_title("STACK (global 0..255)"); axS.axis("off")

# Col 4: Vecteur
axV = fig.add_subplot(gs[:, 4])
fusion_vector = stack_u8.astype(np.float32).reshape(-1)
N = len(fusion_vector)
axV.plot(fusion_vector, np.arange(N), linewidth=0.6, color="k", zorder=3)
axV.invert_yaxis()
axV.set_title(f"Vector (len={N})")
axV.set_xlabel("Value"); axV.set_ylabel("Index")
axV.yaxis.tick_right(); axV.yaxis.set_label_position("right")
axV.set_xlim(0, 255); axV.set_ylim(N-1, 0)

# ===== Couleurs CONSISTANTES pour stack + bandeau + labels =====
colors = ["#e41a1c",  # rouge
          "#377eb8",  # bleu
          "#4daf4a",  # vert
          "#984ea3",  # violet
          "#ffff33"]  # jaune
colors = colors[:n]   # tronque si besoin

# indices des blocs
block_len = int(min_h * min_w)
starts = np.arange(0, n*block_len, block_len)
ends   = starts + block_len - 1
mid_y  = (starts + ends) / 2.0

# bandes sous la courbe (vecteur) avec la PALETTE ci-dessus
for s, e, col in zip(starts, ends, colors):
    axV.axhspan(s, e, facecolor=col, edgecolor=col, alpha=0.18, zorder=1)

# ticks y lisibles et décalés
axV.set_yticks(mid_y)
axV.set_yticklabels([f"{titles[i]} [{starts[i]}:{ends[i]}]" for i in range(n)], fontsize=8)
axV.tick_params(axis="y", pad=25)  # décale les labels vers la droite

# séparateurs & CADRES sur le STACK
H_stack, W_stack = stack_u8.shape
for i in range(1, n):
    y_sep = i * min_h - 0.5
    axS.hlines(y_sep, -0.5, W_stack-0.5, colors="w", linestyles="--", linewidth=1, zorder=3)
for i, col in enumerate(colors):
    y0 = i * min_h
    axS.add_patch(Rectangle((-0.5, y0-0.5), W_stack, min_h,
                            fill=False, linewidth=2.0, edgecolor=col, zorder=3))

# ---------- Bandeau de couleurs à droite du vecteur ----------
plt.subplots_adjust(right=0.82)
pos = axV.get_position()
band_w = 0.012   # bandeau plus fin
gap    = 0.006   # petit espace
axBand = fig.add_axes([pos.x1 + gap, pos.y0, band_w, pos.height], sharey=axV)
for s, e, col in zip(starts, ends, colors):
    axBand.axhspan(s, e, facecolor=col, edgecolor=col, alpha=1.0)
axBand.set_ylim(N-1, 0); axBand.set_xlim(0, 1)
axBand.yaxis.set_visible(False); axBand.xaxis.set_visible(False)
for spine in axBand.spines.values(): spine.set_visible(False)

# ---------- FLÈCHES ----------
for ax_fm in ax_fms:
    arrow(ax0, ax_fm, color="#1f77b4")                         # Input -> Feature
for af, ar in zip(ax_fms, ax_rms):
    arrow_with_label(af, ar, color="#2ca02c", text="MaxPooling")
for ar in ax_rms:
    arrow(ar, axS, color="#d62728")                            # MaxPool -> STACK
arrow_with_label(axS, axV, color="#9467bd", text="Flatten")    # STACK -> Vector

plt.show()

# utilitaire index -> (map, (row, col))
def index_to_stack_coords(idx: int):
    i_map = idx // block_len
    offs  = idx % block_len
    r_local = offs // min_w
    c_local = offs %  min_w
    return int(i_map), (int(r_local), int(c_local))

# **Instruction 2.5. : Exemple avec les mêmes noyaux dans des Conv2D**

In [ ]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import ConnectionPatch

# --------------------------
# 1) Image
# --------------------------
path = "umons2.jpg"   # <-- adapte le chemin pour changer d'image
img_bgr = cv2.imread(path, 1)
assert img_bgr is not None, f"Image introuvable: {path}"
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY).astype(np.float32) / 255.0

device = "cuda" if torch.cuda.is_available() else "cpu"
img_t = torch.from_numpy(img_gray).unsqueeze(0).unsqueeze(0).to(device)  # [1,1,H,W]

# --------------------------
# 2) Définitions de convolutions, normalisation, noyaux
# --------------------------
def conv_from_kernel(kernel, padding, device):
    # Accepte kernel en np.ndarray, list ou torch.Tensor
    if isinstance(kernel, torch.Tensor):
        k = kernel.detach().clone().to(torch.float32).to(device)
    elif isinstance(kernel, np.ndarray):
        k = torch.from_numpy(kernel.astype(np.float32)).to(device)
    else:
        k = torch.tensor(kernel, dtype=torch.float32, device=device)

    conv = nn.Conv2d(1, 1, kernel_size=k.shape, padding=padding, bias=False).to(device)
    with torch.no_grad():
        conv.weight[:] = k.view(1, 1, *k.shape)
    for p in conv.parameters():
        p.requires_grad_(False)
    return conv

def gaussian_kernel2d(ksize, sigma, device):
    ax = torch.arange(ksize, device=device) - (ksize - 1) / 2
    xx, yy = torch.meshgrid(ax, ax, indexing="ij")
    k = torch.exp(-(xx**2 + yy**2) / (2*sigma**2))
    return k / k.sum()

def sobel5x5(device):
    d = torch.tensor([ 1,  2, 0, -2, -1], dtype=torch.float32, device=device)
    s = torch.tensor([ 1,  4, 6,  4,  1], dtype=torch.float32, device=device)
    kx = torch.outer(s, d)
    ky = torch.outer(d, s)
    kx = kx / kx.abs().sum()
    ky = ky / ky.abs().sum()
    return kx, ky

def norm01(t):
    a = t.detach().cpu().squeeze().numpy()
    mn, mx = a.min(), a.max()
    if mx - mn < 1e-12: return np.zeros_like(a)
    return (a - mn) / (mx - mn)

# --------------------------
# 3) 5 filtres fixes (kernels explicites)
# --------------------------
# (A) Gauss 3x3
conv_gauss3 = conv_from_kernel([[1/16, 2/16, 1/16],
                                [2/16, 4/16, 2/16],
                                [1/16, 2/16, 1/16]], padding=1, device=device)

# (B) Gauss 15x15
g15 = gaussian_kernel2d(15, sigma=3.0, device=device)
conv_gauss15 = conv_from_kernel(g15, padding=7, device=device)

# (C) Sobel 5x5 X, (D) Sobel 5x5 Y
kx, ky = sobel5x5(device)
conv_sobelx5 = conv_from_kernel(kx, padding=2, device=device)
conv_sobely5 = conv_from_kernel(ky, padding=2, device=device)

# (E) Laplacien 3x3
conv_lap3 = conv_from_kernel([[0, -1,  0],
                              [-1, 4, -1],
                              [0, -1,  0]], padding=1, device=device)

# --------------------------
# 4) Forward
# --------------------------
feature_maps = [
    conv_gauss3(img_t),
    conv_gauss15(img_t),
    conv_sobelx5(img_t),
    conv_sobely5(img_t),
    conv_lap3(img_t),
]
titles = ["Gaussian 3x3", "Gaussian 15x15", "Sobel X 5x5", "Sobel Y 5x5", "Laplacian 3x3"]

pool = nn.MaxPool2d(2, 2)
reduced_maps = [pool(fm) for fm in feature_maps]   # liste de [1,1,h_i,w_i]

# --------------------------
# 5) Layout + affichage
# --------------------------
rows = len(feature_maps)
fig = plt.figure(figsize=(18, 12))
gs = gridspec.GridSpec(rows, 5, width_ratios=[1.2, 1.0, 1.0, 1.2, 0.8], wspace=0.45, hspace=0.25)

# Col 0: Input (span 5 rows)
ax_input = fig.add_subplot(gs[:, 0])
ax_input.imshow(img_rgb)
ax_input.set_title("Input")
ax_input.axis("off")

# Col 1: Feature maps
ax_fms = []
for i in range(rows):
    ax = fig.add_subplot(gs[i, 1])
    ax.imshow(norm01(feature_maps[i]), cmap="gray")
    ax.set_title("Conv2d avec " + titles[i], fontsize=9)
    ax.axis("off")
    ax_fms.append(ax)

# Col 2: Reduced maps
ax_rms = []
for i in range(rows):
    ax = fig.add_subplot(gs[i, 2])
    ax.imshow(norm01(reduced_maps[i]), cmap="gray")
    ax.set_title(f"MaxPool {i+1}", fontsize=9)
    ax.axis("off")
    ax_rms.append(ax)

# --------------------------
# 6) STACK des reduced alignées, puis vecteur = flatten(STACK)
# --------------------------
# Aligner toutes les reduced maps à la taille minimale (pour empilement propre)
sizes = [rm.shape[-2:] for rm in reduced_maps]     # [(h_i, w_i)]
min_h = min(s[0] for s in sizes)
min_w = min(s[1] for s in sizes)

reduced_aligned = [
    F.interpolate(rm, size=(min_h, min_w), mode="bilinear", align_corners=False)
    for rm in reduced_maps
]  # liste de [1,1,min_h,min_w]

# Col 3: afficher l'empilement (span toutes les lignes)
ax_stack = fig.add_subplot(gs[:, 3])

# Normaliser chaque carte séparément AVANT empilement
reduced_aligned_norm = []
for rm in reduced_aligned:
    a = rm.detach().cpu()
    mn, mx = a.min(), a.max()
    if mx - mn < 1e-12:
        rm_n = torch.zeros_like(a)
    else:
        rm_n = (a - mn) / (mx - mn)
    reduced_aligned_norm.append(rm_n.to(device))

stack_viz = torch.cat(reduced_aligned_norm, dim=-2)  # [1,1, rows*min_h, min_w]

ax_stack.imshow(stack_viz.squeeze().cpu().numpy(), cmap="gray", aspect="auto")
ax_stack.set_title(f"Stack des reduced (alignées)\nsize={tuple(stack_viz.shape[-2:])}")
ax_stack.axis("off")

# Vecteur = flatten(stack_viz)
fusion_vector = stack_viz.flatten(start_dim=1).squeeze(0).cpu().numpy()
N = fusion_vector.size

# Col 4: vecteur vertical
ax_vec = fig.add_subplot(gs[:, 4])
y_idx = np.arange(N)
ax_vec.plot(fusion_vector, y_idx, linewidth=0.8, color='black')
ax_vec.invert_yaxis()
ax_vec.set_title(f"Fusion Vector = flatten(stack)\nlen={N}")
ax_vec.set_xlabel("Value"); ax_vec.set_ylabel("Index")

# ====== Bandeaux & correspondances couleurs ======
import matplotlib as mpl
from matplotlib.patches import Rectangle

# Palette MANUELLE (5 couleurs distinctes, sans orange)
colors = ["#e41a1c",  # rouge
          "#377eb8",  # bleu
          "#4daf4a",  # vert
          "#984ea3",  # violet
          "#ffff33"]  # jaune
colors = colors[:rows]

# Indices de blocs du vecteur (un bloc = une carte alignée)
block_len = int(min_h * min_w)
starts = np.arange(0, rows * block_len, block_len)
ends   = starts + block_len - 1
mid_y  = (starts + ends) / 2.0

# Bandes colorées SOUS la courbe du vecteur
for s, e, col in zip(starts, ends, colors):
    ax_vec.axhspan(s, e, facecolor=col, edgecolor=col, alpha=0.18, zorder=1)

# Bornes et ticks Y (on ne mettra PAS les labels ici à gauche)
ax_vec.set_ylim(N-1, 0)
ax_vec.set_xlim(0, 1.05 * max(1e-6, np.nanmax(fusion_vector)))
ax_vec.set_yticks(mid_y)
ax_vec.set_yticklabels([])                 # <- pas de labels à gauche
ax_vec.tick_params(axis="y", left=False, labelleft=False, pad=0)
ax_vec.set_ylabel("")                      # <- pas de label d'axe à gauche

# Séparateurs + CADRES colorés sur l'image du STACK
H_stack, W_stack = stack_viz.shape[-2], stack_viz.shape[-1]
for i in range(1, rows):
    y_sep = i * min_h - 0.5
    ax_stack.hlines(y_sep, -0.5, W_stack-0.5, colors="w", linestyles="--", linewidth=1, zorder=3)
for i, col in enumerate(colors):
    y0 = i * min_h
    ax_stack.add_patch(Rectangle((-0.5, y0-0.5), W_stack, min_h,
                                 fill=False, linewidth=2.0, edgecolor=col, zorder=3))

# Bandeau de couleurs à DROITE du vecteur + labels à droite du bandeau
plt.subplots_adjust(right=0.87)             # libère de la marge plus à droite
pos = ax_vec.get_position()
band_w = 0.012                              # largeur bandeau
gap    = 0.006                              # espace entre vecteur et bandeau
label_gap = 0.006                           # espace entre bandeau et labels

# Bandeau coloré
axBand = fig.add_axes([pos.x1 + gap, pos.y0, band_w, pos.height], sharey=ax_vec)
for s, e, col in zip(starts, ends, colors):
    axBand.axhspan(s, e, facecolor=col, edgecolor=col, alpha=1.0)
axBand.set_ylim(N-1, 0)
axBand.set_xlim(0, 1)
axBand.yaxis.set_visible(False)
axBand.xaxis.set_visible(False)
for spine in axBand.spines.values():
    spine.set_visible(False)

# Axe factice pour les labels (à droite du bandeau)
axLab = fig.add_axes([pos.x1 + gap + band_w + label_gap, pos.y0, 0.001, pos.height], sharey=ax_vec)
axLab.set_ylim(N-1, 0)
axLab.set_xlim(0, 1)
axLab.set_xticks([])
axLab.set_yticks(mid_y)
axLab.set_yticklabels([f"{titles[i]} [{starts[i]}:{ends[i]}]" for i in range(rows)],
                      fontsize=8, ha="left")
axLab.yaxis.tick_right()
axLab.yaxis.set_label_position("right")
axLab.set_ylabel("Index", rotation=270, labelpad=12, va="center")  # label d'axe à droite
for spine in axLab.spines.values():
    spine.set_visible(False)

# --------------------------
# 7) Flèches
# --------------------------
def arrow_with_label(ax_from, ax_to, text="", color="black", lw=1.4, text_offset=(0,0.05)):
    con = ConnectionPatch(
        xyA=(1.0, 0.5), coordsA=ax_from.transAxes,
        xyB=(0.0, 0.5), coordsB=ax_to.transAxes,
        arrowstyle="->", shrinkA=0, shrinkB=0,
        mutation_scale=12, lw=lw, color=color,
    )
    ax_to.add_artist(con)
    fig2 = ax_from.figure
    fbb = ax_from.get_position(); tbb = ax_to.get_position()
    x0, y0 = fbb.x1, (fbb.y0+fbb.y1)/2
    x1, y1 = tbb.x0, (tbb.y0+tbb.y1)/2
    xm, ym = (x0+x1)/2, (y0+y1)/2
    fig2.text(xm + text_offset[0], ym + text_offset[1], text,
              ha="center", va="center", fontsize=9, color=color)

def arrow_between_axes(ax_from, ax_to, color="black", lw=1.4):
    con = ConnectionPatch(
        xyA=(1.0, 0.5), coordsA=ax_from.transAxes,
        xyB=(0.0, 0.5), coordsB=ax_to.transAxes,
        arrowstyle="->", shrinkA=0, shrinkB=0,
        mutation_scale=12, lw=lw, color=color,
    )
    ax_to.add_artist(con)

# input -> features
for ax_fm in ax_fms:
    arrow_between_axes(ax_input, ax_fm, color="#1f77b4")
# features -> reduced
for af, ar in zip(ax_fms, ax_rms):
    arrow_with_label(af, ar, color="#2ca02c", text="MaxPooling")
# reduced -> stack
for ar in ax_rms:
    arrow_between_axes(ar, ax_stack, color="#d62728")
# stack -> vecteur
arrow_with_label(ax_stack, ax_vec, color="#9467bd", text="Flatten")

plt.show()

# --------------------------
# 8) Infos
# --------------------------
print("Input:", tuple(img_t.shape))
print("Feature maps:", [tuple(fm.shape) for fm in feature_maps])
print("Reduced maps:", [tuple(rm.shape) for rm in reduced_maps])
print("Stack image (HxW):", tuple(stack_viz.shape[-2:]))
print("Fusion vector len:", N)